# Data Science 1 '22/'23

## Numerical Analysis & Signal Analysis - Numerical Differentiation

The following material is covered in Chapter 5 - *Numerical Differentiation* of the book *Numerical methods in engineering with Python 3* by Jaan Kiusalaas.

<img src="https://m.media-amazon.com/images/I/51uvB6TEd5L.jpg" alt="Book cover" height="10%" width="10%" halign="center" />

### Introduction

Numerical differentiation deals with the following problem: We are given the function $y = f(x)$ and wish to obtain one of its derivatives at the point $x = x_k$. The term "given" means that we either have an algorithm for computing the function, or we possess a set of discrete datapoints $(x_i, y_i)$ for $i = 0,1,...,n$. In either case, we have access to a finite number of $(x, y)$ data pairs from which to compute the derivative.

Numerical differentiation is not a particularly accurate process. It suffers from a conflict between roundoff errors (caused by limited machine precision) and errors inherent in interpolation. For this reason, a derivative of a function can never be computed with the same precision as the function itself.

### Intermezzo: Floating Point Arithmetic

This material is partly covered in Section 15 - *Floating Point Arithmetic: Issues and Limitations* of the [Python documentation](https://docs.python.org/3/tutorial/floatingpoint.html).

We assume that you are familiar with [binary](https://en.wikipedia.org/wiki/Binary_number) (and [hexadecimal](https://en.wikipedia.org/wiki/Hexadecimal)) representations of integer numbers (see for example these [short](https://www.youtube.com/watch?v=LpuPe81bc2w), [medium](https://www.youtube.com/watch?v=kTcpd4ef2lU), and [long](https://www.youtube.com/watch?v=vpjhJJQLPq4) video's, or this [series](https://www.youtube.com/watch?v=0qjEkh3P9RE&list=PL726AB973C6E39758) of video's).

Floating-point numbers are represented in computer hardware as base 2 (binary) fractions. For example, the *decimal* fraction 0.375 has value $\frac{3}{10} + \frac{7}{100} + \frac{5}{1000}$. In the same way the *binary* fraction 0.011 has value $\frac{0}{2} + \frac{1}{4} + \frac{1}{8}$. These two fractions have identical values ($\frac{3}{8} = 0.375$), the only real difference being that the first is written in base 10 fractional notation, and the second in base 2.

Unfortunately, most decimal fractions cannot be represented exactly as binary fractions. A consequence is that, in general, the decimal floating-point numbers you enter are only approximated by the binary floating-point numbers actually stored in the machine. For example, no matter how many base 2 digits you're willing to use, the decimal value 0.1 cannot be represented exactly as a base 2 fraction. In base 2, $\frac{1}{10}$ is the infinitely repeating fraction 0.00011001100110011001.. Stop at any finite number of bits, and you get an approximation. This is analogous to how in base 10 some other fractions cannot exactly be represented using a finite number of decimals, like $\frac{1}{7} = 0.142857142857..$.

This may have strange consequences. For example, since 0.1 is not exactly $\frac{1}{10}$, summing three values of 0.1 may not yield exactly 0.3. Try it yourself:

In [1]:
x = 0.1
x + x + x == 0.3

False

On most machines today, floats are approximated using a binary fraction with a sign, an integer numerator, and a denominator that is a power of two. Almost all machines today use [IEEE-754](https://en.wikipedia.org/wiki/IEEE_754-1985) floating point arithmetic. Multiple types of floating point numbers are defined that differ in their precision. A common one is the 64-bit float that maps to IEEE-754 [double precision](https://en.wikipedia.org/wiki/Double-precision_floating-point_format). On input the computer strives to convert 0.1 to the closest fraction it can of the form $\frac{J}{2^N}$, where $J$ is an integer. In the case of 1/10, the binary fraction is $\frac{3602879701896397}{2^{55}}$ which is close to but not exactly equal to the true value of 1/10. Python provides tools that may help on those rare occasions when you really do want to know the exact value of a float. The `float.as_integer_ratio()` method expresses the value of a float as a fraction:

In [2]:
numerator, denominator = x.as_integer_ratio()
print(numerator, denominator)

3602879701896397 36028797018963968


In [3]:
numerator / denominator == x

True

The `hex()` method expresses a float in hexadecimal (base 16), again giving the exact value stored by your computer.

In [11]:
print(x.hex())
print(x)

0x1.999999999999ap-4
0.1


When converted from hexadecimal to binary, we would obtain `'0b1.1001100110011001100110011001100110011001100110011010p-4'`, which corresponds exactly with the (rounded) binary expansion given earlier. The `p` indicates binary scientific notation (similar to te `e` in base 10, like in `0.1 == 1e-1`).

We can access the internal 64-bit binary representation directly using the following bit of code magic.

In [39]:
import struct
bits = ''.join(bin(byte).replace('0b', '').rjust(8, '0') for byte in struct.pack('!d', x))
print(bits)

def bin2float(b):
    res = 0
    b = b[::-1]
    for i in range(0,len(b)):
        res += int(b[i])*(2**i)
    return res

def convertToScientific(x, precision=10):
    res = '('
    sign = x[0]
    exp = x[1:12]
    s = '1.'
    mantissa = x[12:12+precision]
    if sign == '0':
        res += '1'
    else:
        res += '-1' 
    res += ') * 1.' + str(mantissa) + " x 2**(" + str(bin2float(exp) - 1023) + ')' 
    return res
    

def float2bin(f):
    [d] = struct.unpack(">Q", struct.pack(">d", f))
    return f'{d:064b}'
 
print(convertToScientific(float2bin(-1.1)))
print(convertToScientific(float2bin(1.1)))
print(convertToScientific(float2bin(x)))



0011111110111001100110011001100110011001100110011001100110011010
(-1) * 1.0001100110 x 2**(0)
(1) * 1.0001100110 x 2**(0)
(1) * 1.1001100110 x 2**(-4)


A double-precision floating point value is encoded in 8 bytes. The 64 bits of this bit-representation can be dissected into three parts:

* 1 sign bit: `0` (for positive numbers, or `1` for negative numbers)

* 11 exponent bits: `01111111011` (for a biased exponent of 1019)

* 52 mantissa bits: `1001100110011001100110011001100110011001100110011010`

This generally translates into a value

$$
(-1)^\text{sign} \cdot (1.\text{mantissa})_\text{b} \times 2^{\text{exponent}-1023}
$$

Note that in binary, the first digit in scientific notation must always be a 1, so we do not need to encode that explicitly and simply prepend it before the decimal point before the mantissa. In the given case of 0.1, this becomes $(-1)^0 \cdot (1.1001100110011001100110011001100110011001100110011010)_b \times 2^{-4}$ which approximately equals 0.1.

**Exercise 1**

Determine the bit-representations of the following numbers and verify that you understand how these relate to the value of the number itself.

* `1.0`
* `1.5`
* `2.0`
* `-2.0`
* `-1.3333333333333333`
* `3.1415926535897932`
* `1.0000000000000002`
* `1.0000000000000001`

Also try the below "special" numbers.

* `0.0`
* `-0.0`
* `float('inf')`
* `float('-inf')`
* `float('nan')`

Importantly, the number of bits in the mantissa determines the number of accurate decimals. In this case, that leads to $\log_{10}(2) \cdot 52 \approx 16$ accurate decimals in base 10. At the same time, the exponent in combination with its bias determines the magnitude of the largest and smallest numbers that can be represented.

**Exercise 2**

The IEEE-754 standard also includes a [single-precision](https://en.wikipedia.org/wiki/Single-precision_floating-point_format) floating point number of 4 bytes, including 1 sign bit, 8 exponent bits with exponent bias -127, and 23 mantissa bits. Determine the bit-representation of the single-precision number `3.0`. How many correct decimals does this representation approximately have? Hint: first have a look at the double-precision representation, and adapt this form.

Now that we have some idea how numbers are represented in a computer, we continue with the main topic of this lesson: numerical differentiation.

### Difference Approximations

Any function $f$ that is sufficiently well-behaved (i.e. smooth, continuous, differentiable functions) can be approximated locally by means of a polynomial expansion. That is to say, if at some value $x$ the function has a value $y = f(x)$, then for small values of a variable $h$ the function value $f(x+h)$ can be estimated by a polynomial. When using a 0-th order polynomial (i.e. a constant function), then we arrive at $f(x+h) \approx y$. Therefore, the value of the function at $x+h$ is close to the value of the function at $x$. However, we can do better. When using a 1-st order polynomial (i.e. a linear function) we arrive at $f(x+h) = a + b h$. It turns out that the slope $b$ precisely equals the derivative of the function at $x$. Thus, we obtain

$$
f(x+h) = f(x) + f'(x) \cdot h
$$

We can use the above 1-st order [Taylor series](https://en.wikipedia.org/wiki/Taylor_series) to derive an expression for the derivative $f'(x)$.

$$
f'(x) = \frac{f(x+h) - f(x)}{h}
$$

This is called the *forward difference approximation*.

Alternatively, we could have used a negative value for $h$, leading to $f(x-h) = f(x) - f'(x) \cdot h$ and the *backward difference approximation*

$$
f'(x) = \frac{f(x) - f(x-h)}{h}
$$

Both approximations can also be combined to obtain

$$
f'(x) = \frac{f(x+h) - f(x-h)}{2h}
$$

This is called the *central difference approximation*.

**Exercise 3**

Complete the below functions `forward_derivative()`, `backward_derivative()` and `central_derivative()` that calculate the derivative of a provided function `f` at an argument `x` using the forward, backward, and central difference approximations. Replace the default value of the optional parameter `h` with a suitable value.

In [43]:
def forward_derivative(f, x, h=...):
    """df = forward_derivative(f, x, h).
    Calculates the forward difference approximation of the
    function f(x).
    """
    df = (f(x+h)-f(x))/h
    return df

def backward_derivative(f, x, h=...):
    """df = forward_derivative(f, x, h).
    Calculates the backward difference approximation of the
    function f(x).
    """
    df = (f(x)-f(x-h))/h
    return df

def central_derivative(f, x, h=...):
    """df = central_derivative(f, x, h).
    Calculates the central difference approximation of the
    function f(x).
    """
    df = (f(x+h)-f(x-h))/(2*h)
    return df

Below, we apply the forward, backward, and central approximation methods to calculate the derivative of the natural logarithm at $x=2$ using various $h$. The analytical derivative equals $f'(2)=\frac{1}{2}$. We determine the deviation of the numerical outcome from this exact answer for different choices of the parameter $h$.

In [44]:
# Example: derivative of ln(x) at x = 2
from math import log

hs = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8]
df_exact = 0.5

for h in hs:
    df_forward = forward_derivative(log, 2.0, h)
    df_backward = backward_derivative(log, 2.0, h)
    df_central = central_derivative(log, 2.0, h)
    print(f'• h = {h:7.1e}:')
    print(f'  ◦ forward : {df_forward:12.6e} (error={df_forward-df_exact:7.1e})')
    print(f'  ◦ backward: {df_backward:12.6e} (error={df_backward-df_exact:7.1e})')
    print(f'  ◦ central : {df_central:12.6e} (error={df_central-df_exact:7.1e})')

• h = 1.0e-01:
  ◦ forward : 4.879016e-01 (error=-1.2e-02)
  ◦ backward: 5.129329e-01 (error=1.3e-02)
  ◦ central : 5.004173e-01 (error=4.2e-04)
• h = 1.0e-02:
  ◦ forward : 4.987542e-01 (error=-1.2e-03)
  ◦ backward: 5.012542e-01 (error=1.3e-03)
  ◦ central : 5.000042e-01 (error=4.2e-06)
• h = 1.0e-03:
  ◦ forward : 4.998750e-01 (error=-1.2e-04)
  ◦ backward: 5.001250e-01 (error=1.3e-04)
  ◦ central : 5.000000e-01 (error=4.2e-08)
• h = 1.0e-04:
  ◦ forward : 4.999875e-01 (error=-1.2e-05)
  ◦ backward: 5.000125e-01 (error=1.3e-05)
  ◦ central : 5.000000e-01 (error=4.2e-10)
• h = 1.0e-05:
  ◦ forward : 4.999988e-01 (error=-1.2e-06)
  ◦ backward: 5.000013e-01 (error=1.3e-06)
  ◦ central : 5.000000e-01 (error=8.8e-12)
• h = 1.0e-06:
  ◦ forward : 4.999999e-01 (error=-1.2e-07)
  ◦ backward: 5.000001e-01 (error=1.2e-07)
  ◦ central : 5.000000e-01 (error=1.4e-11)
• h = 1.0e-07:
  ◦ forward : 5.000000e-01 (error=-1.3e-08)
  ◦ backward: 5.000000e-01 (error=1.3e-08)
  ◦ central : 5.000000e-01 (

Have a look at the size of the errors. Importantly, if we decrease the magnitude of $h$ by a factor $n$, then the error of the `forward_derivative` and `backward_derivative` methods also decreases by a factor $n$, but the error of the `central_derivative` method decreases by a factor $n^2$. We say that the `forward_derivative` and `backward_derivative` methods have an error of order $\mathcal{O}(h)$, whereas the `central_derivative` method has an error of order $\mathcal{O}(h^2)$. This is often written as

$$
\begin{aligned}
f'(x) &= \frac{f(x+h) - f(x)}{h} + \mathcal{O}(h)
\\
f'(x) &= \frac{f(x) - f(x-h)}{h} + \mathcal{O}(h)
\\
f'(x) &= \frac{f(x+h) - f(x-h)}{2h} + \mathcal{O}(h^2)
\end{aligned}
$$

For sufficiently small $h$, the `central_derivative` method will therefore tend to be preferable. However, keep in mind that when $h$ becomes too small, numerical round-off errors start to dominate.

### Higher-order derivatives

We can include higher-order terms in the polynomial approximation of the Taylor series expansion. The general form is

$$
f(x+h) = \sum_{n=0}^N \frac{1}{n!} f^{(n)}(x) \cdot h^n + \mathcal{O}(h^{N+1})
$$

We will limit ourselves to the case $N=2$, for which we get the quadratic form

$$
f(x+h) = f(x) + f'(x) \cdot h + \frac{1}{2} f''(x) \cdot h^2 + \mathcal{O}(h^3)
$$

If we apply this to arguments $x-h$, $x$, and $x+h$, and disregard the error term, we obtain a system of three equations

$$
\begin{aligned}
f(x-h) &= f(x) - f'(x) \cdot h + \frac{1}{2} f''(x) \cdot h^2
\\
f(x) &= f(x)
\\
f(x+h) &= f(x) + f'(x) \cdot h + \frac{1}{2} f''(x) \cdot h^2
\end{aligned}
$$

that can be solved using linear algebra techniques to find the solution

$$
\begin{aligned}
f(x) &= f(x)
\\
f'(x) &= \frac{f(x+h) - f(x-h)}{2h}
\\
f''(x) &= \frac{f(x+h) - 2f(x) + f(x-h)}{h^2}
\end{aligned}
$$

We recognise the central difference approximation for $f'(x)$.

**Exercise 4**

Complete the below function `central_derivative2()` that computes the second derivative of a provided function `f`. Determine the order of the error in $f''(x)$ by comparing the numerical value of $f''(x)$ for $f(x)=\ln(x)$ at $x=2$ to its exact analytical value.

In [45]:
def central_derivative2(f, x, h=...):
    """df = central_derivative2(f, x, h).
    Calculates the second-order derivative of the
    function f(x).
    """
    ddf = (f(x+h)-2*f(x)+f(x-h))/(h*h)
    return ddf

In [46]:
# Example: second derivative of ln(x) at x = 2
ddf_exact = -0.25

for h in hs:
    ddf_central = central_derivative2(log, 2.0, h)
    print(f'h = {h:7.1e}:')
    print(f'\t* central : {ddf_central:12.6e} (error={ddf_central-ddf_exact:7.1e})')

h = 1.0e-01:
	* central : -2.503130e-01 (error=-3.1e-04)
h = 1.0e-02:
	* central : -2.500031e-01 (error=-3.1e-06)
h = 1.0e-03:
	* central : -2.500000e-01 (error=-3.1e-08)
h = 1.0e-04:
	* central : -2.500000e-01 (error=1.3e-08)
h = 1.0e-05:
	* central : -2.499989e-01 (error=1.1e-06)
h = 1.0e-06:
	* central : -2.499112e-01 (error=8.9e-05)
h = 1.0e-07:
	* central : -2.553513e-01 (error=-5.4e-03)
h = 1.0e-08:
	* central : 0.000000e+00 (error=2.5e-01)


### Exercises

**Exercise 5**

Determine the bit-representations of `0.1 + 0.2 + 0.3` and `0.6` and confirm that these are not identical. Are `1.0 + 2.0 + 3.0` and `6.0` different as well? Explain your observations.

**Exercise 6**

Compute the first- and second-order derivatives of the exponential function $g(x) = 2^x$ at $x = 0$ using the following three methods:

* analytically, using symbolic differentiation;

* using your own functions `central_derivative()` and `central_derivative2()`;

* using the function `derivative` of the module `scipy.misc` (see the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.misc.derivative.html)).

**Exercise 7**

Compute the derivative of the cosine- and sine-functions at $x = \pi$ using the forward, backward, and central difference approximations. Also, investigate the order of the errors; explain any deviations that you observe in behaviour of the error compared to those in previous examples.

**Exercise 8**

If we sample a function $f$ at four positions ⁠— say at $f(x-2h)$, $f(x-h)$, $f(x+h)$, and $f(x+2h)$ ⁠— then we may use a third-order Taylor expansion

$$
f(x+h) = f(x) + f'(x) \cdot h + \frac{1}{2} f''(x) \cdot h^2 + \frac{1}{6} f'''(x) \cdot h^3 + \mathcal{O}(h^4)
$$

to estimate all derivatives up to third order.

Find approximating expressions for $f(x)$, $f'(x)$, $f''(x)$, and $f'''(x)$ in terms of $f(x-2h)$, $f(x-h)$, $f(x+h)$, and $f(x+2h)$ and implement these expressions. Estimate the various derivatives of $f(x)=\ln(x)$ for various values of $h$ and compare them to their theoretical values.

Based on the behavior of these errors, what are the orders $\mathcal{O}(h^n)$ in these approximations? Are these expressions better than the earlier forward/backward and central derivatives?

***